# Alpaca-py trading multi-leg (mleg) options

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/options/options-trading-mleg.ipynb)

- This notebook shows how to use alpaca-py with options trading API endpoints to place multi-leg options orders.
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for options as an example.

In [ ]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/

api_key = None
secret_key = None

#### We use paper environment for this example ####
paper = True  # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables

trade_api_url = None

In [ ]:
import os

if api_key is None:
    api_key = os.environ.get('ALPACA_API_KEY')

if secret_key is None:
    secret_key = os.environ.get('ALPACA_SECRET_KEY')

In [ ]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    !python3 -m pip install alpaca-py
    import alpaca

In [ ]:

from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import calendar

from alpaca.trading.client import TradingClient

from alpaca.trading.requests import (
    GetOptionContractsRequest,
    MarketOrderRequest,
    OptionLegRequest,
    ReplaceOrderRequest,
    LimitOrderRequest,
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderStatus,
    TimeInForce,
    OrderClass,
    ContractType
)
from alpaca.common.exceptions import APIError

## Trading Client

In [ ]:
# setup client
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [ ]:
# First, let us see if the account is enabled for multi-leg options trading
acct = trade_client.get_account()
if acct.options_trading_level >= 3:
    print("Account is enabled for multi-leg options trading!")
else:
    print("Account is not enabled for multi-leg options trading!")

### The Straddle

A straddle is an options strategy that involves buying both a call and a put option with the same strike price and expiration date. This is typically done when the trader expects a large price movement but is unsure of the direction. That is to say, the trader will profit if the price moves significantly in either direction from the strike price. By using a multi-leg order, rather than place two separate orders, the trader can ensure that both legs
of the order are executed simultaneously.

In [ ]:
# Let us find the earnings date for TSLA closest to today
# Typically these occur OCT, JAN, APR, or JUL around the middle of the 3rd week of the month


today = datetime.now(tz=ZoneInfo("America/New_York"))

earnings_month = None
earnings_day = None
earnings_year = today.year

# find next earnings date month
if today.month < 4:
    earnings_month = 4
elif today.month < 7:
    earnings_month = 7
elif today.month < 10:
    earnings_month = 10
else:
    earnings_year += 1
    earnings_month = 1

# find wednesday of the 3rd week of the month
earnings_day = calendar.monthcalendar(earnings_year, earnings_month)[-2][calendar.WEDNESDAY]

earnings_date = datetime(earnings_year, earnings_month, earnings_day)

earnings_date

In [ ]:
# This is a function that will return a contract which minimizes the difference from a target price
def find_nearest_strike_contract(contracts, target_price):
    min_diff = 0
    min_contract = None
    for contract in contracts:
        diff = abs(float(contract.strike_price) - target_price)
        if min_contract is None or diff < min_diff:
            min_diff = diff
            min_contract = contract
    return min_contract

In [ ]:
# Let us find a call and put option for TSLA with the closest expiration date to the earnings date
# and a strike price closest to some price
order_legs = []
optimal_price = 413.82  # this could be our entry price, for example, that we wish to hedge against (if we expect volatility)

for c_type in [ContractType.CALL, ContractType.PUT]:
    req = GetOptionContractsRequest(
        underlying_symbols=["TSLA"],
        status=AssetStatus.ACTIVE,
        expiration_date_gte=earnings_date,
        expiration_date_lte=earnings_date + timedelta(weeks=4),
        style=ExerciseStyle.AMERICAN,
        strike_price_gte=str(optimal_price - 20),
        strike_price_lte=str(optimal_price + 20),
        limit=10,
        type=c_type,  # We could do this in one request, without setting type, if we set up the
        # gte, lte, and limit strategically (but the current approach is a bit more generalizable).
        # See the next example for how to do this.
        page_token=None,
    )
    cts = trade_client.get_option_contracts(req)

    c = find_nearest_strike_contract(cts.option_contracts, optimal_price)
    order_legs.append(OptionLegRequest(
        symbol=c.symbol,
        side=OrderSide.BUY,
        ratio_qty=1
    ))

# We should see that the symbols are similar, like "TSLA______C________" and "TSLA______P________",
# with all values marked as "_" being the same in both symbols.
# Such is because we expect only the contract type (call or put, C or P) to be different.
order_legs

In [ ]:
# Now we place the order for both legs at the same time
req = MarketOrderRequest(
    qty=1,
    order_class=OrderClass.MLEG,
    time_in_force=TimeInForce.DAY,
    legs=order_legs
)
res = trade_client.submit_order(req)
res

In [ ]:
# Note that we can query via order ids or client order ids for the whole order or individual legs

# Query by the order's client id
q1 = trade_client.get_order_by_client_id(res.client_order_id)

# Query by the whole order's id
q2 = trade_client.get_order_by_id(res.id)

# Query just the first leg's client id
q3 = trade_client.get_order_by_client_id(res.legs[0].client_order_id)

# Query by first leg's id
q4 = trade_client.get_order_by_id(res.legs[0].id)

print(f"Q1: {q1}\n===\nQ2: {q2}\n===\nQ3: {q3}\n===\nQ4: {q4}")

### The Iron Condor

This is traditionally harder to implement because it requires 4 legs. However, with multi-leg orders, Alpaca makes doing this a breeze! Moreover, we can use a limit order on the multi-leg order to better time our entry (which is much more important with an iron condor than with a straddle).

This time we are considering 4 separate strike prices: A, B, C, and D. We buy a put at A, sell a put at B, sell a call at C, and buy a call at D. A trader expects the price to stay within the range of B and C and will typically enter only once the price is between B and C. Using a limit order, we execute the order when the *net* price is 0 or better.



In [ ]:
# At the time of writing, TSLA has a standard deviation (a common measure of variation for the price) of 5.02 and a price of 413.82
# Let's create a interval around the current price such that B and C are a standard deviation away from each other

stddev = 5.02
B = optimal_price - (stddev / 2)
C = optimal_price + (stddev / 2)
A = B - stddev
D = C + stddev

print(f"A: {A}, B: {B}, C: {C}, D: {D}")

In [ ]:
# Now, instead of doing two separate requests, we can do one request to get all the contracts we need
# We set this up by setting the gte, lte, and limit strategically
# That being said, it is still safest to do separate requests as we did in the prior example as the number of contracts
# returned can vary based on the strike prices and expiration dates (which are not fixed in this notebook).

# Let's find the delta between strike prices from our prior example
max_delta = 0
for i in range(len(cts.option_contracts) - 1):
    delta = abs(float(cts.option_contracts[i].strike_price) - float(cts.option_contracts[i + 1].strike_price))
    if delta > max_delta:
        max_delta = delta


def next_divisible(n: float, div: float, round_up: bool):
    if n % div == 0:
        return n
    if round_up:
        return n + div - n % div
    return n - n % div


min_contract_price = next_divisible(A, max_delta, False)
max_contract_price = next_divisible(D, max_delta, True)
print(f"Min: {min_contract_price}, Max: {max_contract_price}")

In [ ]:
# Now the actual request
req = GetOptionContractsRequest(
    underlying_symbols=["TSLA"],
    status=AssetStatus.ACTIVE,
    expiration_date_gte=cts.option_contracts[0].expiration_date - timedelta(weeks=5),
    expiration_date_lte=cts.option_contracts[0].expiration_date - timedelta(weeks=3),
    style=ExerciseStyle.AMERICAN,
    strike_price_gte=str(min_contract_price),
    strike_price_lte=str(max_contract_price),
    page_token=None,
)
cts_m = trade_client.get_option_contracts(req)
cts_m

In [ ]:
# This finds contracts which minimizes the difference from a set of target prices
class ContractBuffer:
    def __init__(self, optimal, is_call, is_buy):
        self.optimal = optimal
        self.is_call = is_call
        self.is_buy = is_buy
        self.contract = None
        self.diff = 0

    def __repr__(self):
        return f"Contract: {self.contract}, Optimal: {self.optimal}, Diff: {self.diff}, Is Call: {self.is_call}"


buffers = [ContractBuffer(A, False, True), ContractBuffer(B, False, False), ContractBuffer(C, True, False),
           ContractBuffer(D, True, True)]

for contract in cts_m.option_contracts:
    for buff in buffers:
        is_call = contract.type == ContractType.CALL
        if buff.is_call != is_call:
            continue
        diff = abs(float(contract.strike_price) - buff.optimal)
        if diff < buff.diff or buff.contract is None:
            buff.diff = diff
            buff.contract = contract

order_legs_m = []
for buff in buffers:
    order_legs_m.append(OptionLegRequest(
        symbol=buff.contract.symbol,
        side=OrderSide.BUY if buff.is_buy else OrderSide.SELL,
        ratio_qty=1
    ))

del buffers
order_legs_m

In [ ]:
# Order for the iron condor
req = LimitOrderRequest(
    qty=50,
    order_class=OrderClass.MLEG,
    time_in_force=TimeInForce.DAY,
    legs=order_legs_m,
    limit_price=0  # i.e., for a net price of 0
)
res = trade_client.submit_order(req)
res

## Querying, Replacing, and Cancelling Multi-leg Orders

In [ ]:
# We have two resolutions at which we can query the order status: by the mleg's id or by the individual legs' ids.

# Query by the order's id
q1 = trade_client.get_order_by_client_id(res.client_order_id)

# Query just the first leg's id
q2 = trade_client.get_order_by_client_id(res.legs[0].client_order_id)

print(f"Query by whole order id: {q1}\n===\nQuery by leg id: {q2}")

In [ ]:
# Note that we cannot cancel nor replace an individual leg
try:
    if q1.legs[0].status != OrderStatus.FILLED:
        res = trade_client.cancel_order_by_id(res.legs[0].id)
        print(f"Cancelled leg: {res}")
    else:
        print("Order is already filled.")
except APIError as e:
    print(f"Error: {e}")

try:
    if q1.status != OrderStatus.FILLED:
        req = ReplaceOrderRequest(
            qty=1
        )
        res = trade_client.replace_order_by_id(res.legs[0].id, req)
        print(f"Replaced order: {res}")
    else:
        print("Order is already filled.")
except APIError as e:
    print(f"Error: {e}")

# Should get an output of form
# Error: {"code":42210000,"message":"cannot cancel individual legs of a mleg order"}
# Error: {"code":42210000,"message":"cannot replace individual legs of a mleg order"}

In [ ]:
# Replace overall order
if q1.status != OrderStatus.FILLED:
    # We can replace the order
    req = ReplaceOrderRequest(
        qty=55,
    )
    res = trade_client.replace_order_by_id(res.id, req)
    print(f"Replaced order: {res}")
else:
    print("Order is already filled.")

In [ ]:
# Cancel the whole order
trade_client.cancel_order_by_id(res.id)

In [ ]:
# To validate, we can query the order again
res = trade_client.get_order_by_client_id(res.client_order_id)
res